In [1]:
import json
import os

import compmusic.dunya.conn
from compmusic.dunya.docserver import get_document_as_json, get_mp3

import urllib


In [2]:
# paste your Dunya token below
token = 'XXX'
compmusic.dunya.conn.set_token(token)


In [3]:
# paths
data_folder = './data/'

symbtr_mbid_url = 'https://raw.githubusercontent.com/MTG/SymbTr/master/symbTr_mbid.json'
symbtr_mbid = json.loads(urllib.urlopen(symbtr_mbid_url).read())

makam_recs_file = './dataset.json'
makam_recs = json.load(open(makam_recs_file))


In [4]:
for makam, recs in makam_recs.items():
    # makam folder
    makam_folder = os.path.join(data_folder, makam)
    
    # create makam directory if it doesn't exist
    if not os.path.exists(makam_folder):
        os.makedirs(makam_folder)
    
    for rec in recs:
        recid = os.path.split(rec)[-1]
        print makam + ', ' + recid
        
        # audio metadata
        metadata = get_document_as_json(recid, 'audioanalysis', 'metadata')
        
        ## ------------------ Obtain the relevant SymbTr score ------------
        # get the symbtr work
        assert len(metadata['works']) == 1, 'More that one work in the recording %s' % recid
        workid = metadata['works'][0]['mbid']

        # get the symbtr name
        symbtr = None
        for sm in symbtr_mbid:
            if workid in sm['uuid']:
                symbtr = sm['name']

        assert symbtr is not None, 'No SymbTr score matched for the work %s' % workid

        # create symbtr folder
        symbtr_folder = os.path.join(makam_folder, symbtr)
        if not os.path.exists(symbtr_folder):
            os.makedirs(symbtr_folder)
            
        # copy the relevant SymbTr documents
        # TODO

        ## ------------------ Obtain the relevant recording and features ------------
        # create audio folder
        rec_folder = os.path.join(symbtr_folder, recid)
        if not os.path.exists(rec_folder):
            os.makedirs(rec_folder)
        
        # metadata 
        metadata_file = os.path.join(rec_folder, 'metadata.json')
        json.dump(metadata, open(metadata_file, 'w'), indent=4)
        
        # ... download the relevant audio analysis features
        # pitch
        pitch = get_document_as_json(recid, 'audioanalysis', 'pitch_filtered')
        pitch_file = os.path.join(rec_folder, 'pitch_filtered.json')
        json.dump(pitch, open(pitch_file, 'w'))

        # mp3
        mp3 = get_mp3(recid)
        mp3_file = os.path.join(rec_folder, recid + '.mp3')
        open(mp3_file, 'w').write(mp3)

        # pitch distribution
        pd = get_document_as_json(recid, 'audioanalysis', 'pitch_distribution')
        pd_file = os.path.join(rec_folder, 'pitch_distribution.json')
        json.dump(pd, open(pd_file, 'w'))

        # stable notes
        nm = get_document_as_json(recid, 'audioanalysis', 'note_models')
        nm_file = os.path.join(rec_folder, 'note_models.json')
        json.dump(nm, open(nm_file, 'w'))
        
        # ... download the relevant joint analysis features
        # TODO


huzzam, ea7e2b41-e009-48f9-88e3-9fcf9854d9f9
huzzam, da4d7b0b-af8f-42ea-bce0-0bda85fa6801
huzzam, fce6fe19-a1cd-4bdc-b04a-dc6d1c2ca8c8
huzzam, fa8a4b53-bafb-467c-84ea-3089c28f9463
huzzam, 9e02612c-5a0f-4d34-a627-773d86eb8532
huzzam, c6089405-6f18-4562-8c24-fff2adc42eee
huzzam, 97eb2ab3-11f9-4ae5-92b3-115e490a11a6
huzzam, 470c6b30-22dd-465e-b695-14aa594c2426
huzzam, 5ad212c5-56a9-4207-b542-102bc3fec2e3
huzzam, 4df2f27a-fca8-4166-9674-250855492187
rast, b466a80f-820e-4083-857f-2417d1551134
rast, d9cdfff9-fe30-4064-b271-f97dbcf6071b
rast, b1e0c3ae-7070-4d80-80f3-2875a3802226
rast, 31bf3d56-03d8-484e-b63c-ae5ae9a6e733
rast, e2e735a4-a034-4bc5-971e-ae5fa5969c7c
rast, 0321f83e-bee5-4009-983b-1b4fe3c89f23
rast, 0967e5b6-8bff-45b3-9926-e9cb3d244463
rast, 7aec9833-6482-4917-87bd-e60c7c1dae3c
rast, 385acc87-87ae-4197-ae8b-2226655ba304
rast, 6c48b93f-16fa-4adc-bf7b-26652f64defb
hicaz, ff9769d3-81f4-4d30-b6a4-30179f668236
hicaz, ecd6dafc-13a9-4852-b45c-31ba7a3a21d5
hicaz, 59dcf72f-301c-41d9-ab5a-e